In [1]:
from azureml.core import Workspace

In [9]:
# Create or retrieve an existing Azure ML workspace
# ws = Workspace.get(name='ai_ml_course')
subscription_id='engbilalk',
resource_group='engbilalk-rg',
location='eastus2'

In [26]:
from azureml.core import Workspace, Model

# First get the workspace reference
ws = Workspace.get(name='ai_ml_course',
                   subscription_id='35883f52-75b2-43c5-ac8c-15e13f9761ee',
                   resource_group='engbilalk-rg',
                  location='eastus2')

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network with one hidden layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128) # input layer (784 input features)
        self.fc2 = nn.Linear(128, 10)  # Output layer (10 classes)
        self.relu = nn.ReLU()          # Activation function
        
    def forward(self, x):
        x = self.relu(self.fc1(x)) # Apply ReLU after the first layer
        x = self.fc2(x)            # Output layer
        return x

In [12]:
# Intialize the neural network
model = SimpleNN()

In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)  #Stochastic Descent

In [14]:
from torchvision import datasets, transforms

In [16]:
# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

In [17]:
# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad() # Reset gradients
        
        # Forward pass
        outputs = model(inputs.view(-1, 784))  # Flatten input
        loss = criterion(outputs, labels)      # Compute loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 0.8818728608290355
Epoch 2, Loss: 0.37548073340654375
Epoch 3, Loss: 0.3219303167919318
Epoch 4, Loss: 0.2920507937649886
Epoch 5, Loss: 0.26955332116087277


In [28]:
from azureml.core import Model

# Save the trained model
torch.save(model.state_dict(), 'simple_nn.pth')

In [29]:
# Register the model
model = Model.register(workspace=ws, 
                       model_path='simple_nn.pth', 
                       model_name='simple_nn')

Registering model simple_nn
